In [217]:
import pandas as pd

data = pd.read_csv("supercasa_catalog.csv")
data.drop_duplicates(inplace= True)
data

,data_id,data_layer,href,value
0,889246,889246,/arrendar-casas/aveiro/gloria-e-vera-cruz,Glória e Vera Cruz (45)
1,12521,889246,/zona/arrendar-casas/forum-aveiro/aveiro/glori...,Forum Aveiro (26)
2,73952,889246,/zona/arrendar-casas/bombeiros-novos/aveiro/gl...,Bombeiros Novos (24)
3,4543,889246,/zona/arrendar-casas/rossio/aveiro/gloria-e-ve...,Rossio (20)
4,4544,889246,/zona/arrendar-casas/beira-mar/aveiro/gloria-e...,Beira Mar (20)
...,...,...,...,...
47449,2704,890082,/zona/arrendar-casas/conservatoria-do-registo-...,Conservatória do Registo Civil e Predial de Sã...
47451,66442,890082,/zona/arrendar-casas/centro-de-saude-sao-pedro...,Centro de Saúde São Pedro do Sul (2)
47453,763,0,/zona/arrendar-casas/praia-fluvial-de-pouves/s...,Praia Fluvial de Pouves (2)
47454,2040,890082,/zona/arrendar-casas/tribunal-judicial-da-coma...,Tribunal Judicial da Comarca de Viseu Unidade ...


In [218]:
data[data["data_id"] == 246]

,data_id,data_layer,href,value
23027,246,157,/zona/arrendar-casas/universidade-de-lisboa-fa...,Universidade de Lisboa - Faculdade de Motricid...
25683,246,0,/zona/arrendar-casas/universidade-de-lisboa-fa...,Universidade de Lisboa - Faculdade de Motricid...
45917,246,246,/arrendar-casas/montalegre,Montalegre (1)


In [219]:
# Function to find all parents
def find_parents(data_id, df):
    parents = []
    current_id = data_id
    while True:
        parent_row = df.loc[df['data_id'] == current_id, 'data_layer']
        if parent_row.empty:
            break
        parent = parent_row.values[0]
        if parent == current_id:
            break
        parents.append(parent)
        current_id = parent
    return parents

# Create a dictionary to store hierarchy
parent_dict = {}
for data_id in data['data_id']:
    parents = find_parents(data_id, data)
    parent_dict[data_id] = parents

# Convert dictionary to DataFrame
hierarchy_data = {'data_layer': [], 'parent_1': [], 'parent_2': [], 'parent_3': []}
for data_id, parents in parent_dict.items():
    hierarchy_data['data_layer'].append(data_id)
    hierarchy_data['parent_1'].append(parents[0] if len(parents) > 0 else None)
    hierarchy_data['parent_2'].append(parents[1] if len(parents) > 1 else None)
    hierarchy_data['parent_3'].append(parents[2] if len(parents) > 2 else None)

hierarchy_df = pd.DataFrame(hierarchy_data)

# Create a function to replace values in parent columns with href
def replace_values_with_href(df, data_df, parent_cols):
    href_lookup = dict(zip(data_df['data_id'], data_df['href']))
    
    for col in parent_cols:
        df[col] = df[col].map(href_lookup).fillna(df[col])
    
    return df

# Create a dataframe from the hierarchical JSON structure
flat_data = []

for _, row in hierarchy_df.iterrows():
    parent_1 = row['parent_1']
    parent_2 = row['parent_2']
    parent_3 = row['parent_3']
    
    if pd.notna(parent_1):
        flat_data.append({
            "parent_1": parent_1,
            "parent_2": parent_2,
            "parent_3": parent_3
        })

# Convert to DataFrame
flat_df = pd.DataFrame(flat_data)

# Drop duplicates
flat_df.drop_duplicates(inplace=True)

# Replace values in parent_1, parent_2, and parent_3 columns
parent_cols = ['parent_1', 'parent_2', 'parent_3']
updated_flat_df = replace_values_with_href(flat_df, data, parent_cols)
updated_flat_df

,parent_1,parent_2,parent_3
0,/arrendar-casas/aveiro/gloria-e-vera-cruz,NaN,NaN
11,0.0,NaN,NaN
29,/arrendar-casas/coimbra,NaN,NaN
32,/arrendar-casas/alcobaca,/zona/arrendar-casas/metro-arroios/lisboa,NaN
34,/zona/arrendar-casas/metro-casa-do-povo/seixal,/zona/arrendar-casas/metro-arroios/lisboa,NaN
...,...,...,...
4921,/arrendar-casas/sao-pedro-do-sul,0.0,NaN
4933,/arrendar-casas/lamego,0.0,NaN
4936,/arrendar-casas/penedono,/arrendar-casas/lisboa/benfica,NaN
4937,/zona/arrendar-casas/instituto-universitario-m...,0.0,NaN


In [220]:
def replace_values_with_href(df, data_df, parent_cols):
    href_lookup = dict(zip(data_df['data_id'], data_df['href']))
    
    for col in parent_cols:
        df[col] = df[col].map(href_lookup).fillna(df[col])
    
    return df

# Replace values in parent_1, parent_2, and parent_3 columns
parent_cols = ['parent_1', 'parent_2', 'parent_3']
updated_flat_df = replace_values_with_href(flat_df, data, parent_cols)